In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
import numpy as np
from pos_tagger import PosTagger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression


In [ ]:
model_url = 'roneneldan/TinyStories-3M'

model = AutoModelForCausalLM.from_pretrained(model_url)
tokenizer = AutoTokenizer.from_pretrained(model_url)

In [ ]:
with open('data/tinystories_val.txt', 'r') as file:
    stories = file.read()
    stories = stories.split("<|endoftext|>\n")

In [ ]:
class miav():
    def __init__(self, model, tokenizer, dataset):
        self.model = model
        self.tokenizer = tokenizer
        self.dataset = dataset
    
    
    def attach_value_hooks(self, model):
        values = []
        def extract_value_hook(module, input, output):
            values.append(output)
        
        value_hook_list = []
        for layer in range(8):
            value_hookpoint = f'transformer.h.{layer}.attn.attention.v_proj'
            value_hook = model.get_submodule(value_hookpoint).register_forward_hook(extract_value_hook)
            value_hook_list.append(value_hook)
        return value_hook_list, values


    def get_values(self, model, tokenizer, inputs:list[str]):
        value_hook_list, values = self.attach_value_hooks(model)
        for input_ in inputs:
            tokenized = tokenizer.encode(input_, return_tensors='pt')
            model.forward(tokenized)
        for value_hook in value_hook_list:
            value_hook.remove()
        return values


idk = miav(model, tokenizer, stories)

In [ ]:
sentence = "I am a cat"

tagger = PosTagger(tokenizer)

In [ ]:
layer_index = 0
head_index = 0

def encode_sentence(sentence:str, tagger, head_index):
    li = idk.get_values(model, tokenizer,[sentence] )
    X_d = defaultdict(list)
    y_d = defaultdict(list)

    for layer_index, layer in enumerate(li):

        for batch in layer:
            _, tags = tagger.tag_input(sentence)
            for attention_token, tags in zip(batch,tags):

                head_value = attention_token[head_index:head_index+8]

                X_d[layer_index].append(head_value.detach().numpy())
                y_d[layer_index].append(tags)

    return X_d, y_d

sentences = ["I am a cat", "i like cheese"]
X_all = []
y_all = []

for sentence in stories[0:100]:
    for i in range(8):
        X, y = encode_sentence(sentence, tagger, head_index)


In [ ]:
import pandas as pd
pd.DataFrame(d)

In [ ]:
li[0].shape

In [ ]:
def create_data(model, tokenizer, layer, head_index, sentences):
    tagger = PosTagger(tokenizer)
    X = []
    Y = []


    for sentence in sentences:

        value_matrix = idk.get_values(model, tokenizer, layer, head_index,[sentence])
        tokens, tags = tagger.tag_input(sentence)
        attention_data = idk.get_value_attention_sentence_level(value_matrix,0)[0]
        for token, tag in zip(attention_data, tags):
            X.append(token.detach().numpy())
            Y.append(tag)
    
    # Initialize the LabelEncoder
    label_encoder = LabelEncoder()
    
    # Fit and transform the data
    Y = label_encoder.fit_transform(Y)
    X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42)

    return X_train, X_test, y_train, y_test
    

def train_model(X_train, X_test, y_train, y_test):
    clf = LogisticRegression(random_state=0).fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    return score

In [ ]:
layer_list =[]
create_data()

for layer in layers:
    head_list = []
    for head in heads:
        score = train_model(X_train, X_test, y_train, y_test)
        head_list.append(score)
    layer_list.append(head_list)